In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy.stats as st
import math

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Context
Online store of sporting goods: clothing, shoes, accessories and sports nutrition.

Content
On the main page of the store they show users banners in order to stimulate their sales. Now one of 5 banners is randomly displayed there. Each banner advertises a specific product or the entire company. Our marketers believe that the experience with banners can vary by segment, and their effectiveness may depend on the characteristics of user behavior.

Assumption: User site version affects user behaviour
* H0 = user site version affects selection
* H1 = User site version doesnt affect selection

confidence level = 0.05 for 95% confidence interval
practical significance = 0.01 

In [ ]:
def abStatistics(n1, n2, prob, alpha, x1,x2,practical_significance, std1, std2):
    SE = math.sqrt(prob * (1-prob) * (1/n1 + 1/n2))
    alpha = st.norm.ppf(1-alpha/2)
    margin_of_error = SE * alpha
    
    mean_diff = x1-x2
    lower_bound = mean_diff-margin_of_error
    upper_bound = mean_diff + margin_of_error
    
    zscore = mean_diff / math.sqrt((math.pow(std1,2)/n1) + (math.pow(std2,2)/n2))
    pvalue = st.norm.sf(abs(zscore))*2
    d = (std1-std2)/SE
    if practical_significance < lower_bound:
        print("Reject null Hypothesis")
    else:
        print ("Accept null Hypothesis")
        
    if pvalue <= alpha:
        print("Reject null Hypothesis, Result is statistical significant")
    else:
        print("Accept Alternate Hypothesis, Result not statistical significant ")

    print(f"Standard Error: {SE}, margin of error: {margin_of_error},\n CI ({lower_bound},{upper_bound})")

In [ ]:
dataset = pd.read_csv("/kaggle/input/how-to-do-product-analytics/product.csv")
dataset.select_dtypes('object').nunique()

In [ ]:
dat = dataset.groupby(['site_version', 'title']).agg({'target':"sum", 'user_id':"count"}).reset_index()
dat
#dat['cal'] = 0
#dat['cal']=dat['target'].apply(lambda x: x/(dat['user_id'].sum())


1. Compute data for desktop orders

In [ ]:
total_desktop = dataset['target'][dataset['site_version'] == "desktop"].count()
total_desktop_order = dataset['target'][(dataset['site_version'] == "desktop") & (dataset['title'] == "order")].sum()
mean_desktop_order = total_desktop_order/total_desktop
std_desktop_order = dataset['target'][dataset['site_version'] == "desktop"].std()

2. Compute data for mobile orders

In [ ]:
total_mobile = dataset['target'][dataset['site_version'] == "mobile"].count()
total_mobile_order = dataset['target'][(dataset['site_version'] == "mobile") & (dataset['title'] == "order")].sum()
mean_mobile_order = total_mobile_order/total_mobile
std_mobile_order = dataset['target'][dataset['site_version'] == "mobile"].std()

3. Probability of Order

In [ ]:
prob = (total_desktop_order + total_mobile_order)/ (total_mobile + total_desktop)

4. Calculate the statistics

In [ ]:
alpha = 0.05 #95%  confidence level
pratical_sig = 0.01
abStatistics(total_desktop, total_mobile, prob, alpha, mean_desktop_order,mean_mobile_order,pratical_sig, std_desktop_order, std_mobile_order)